# Registered Interest and Principal of Securities (STRIPS) Bonds

## Background
[Registered Interest and Principal of Securities (STRIPS) bonds](https://en.wikipedia.org/wiki/United_States_Treasury_security#STRIPS) are a unique type of fixed-income investment instrument that provides investors with an alternative way to access the income and coupon payments of Treasury securities. STRIPS bonds are created by separating a Treasury security’s coupon and principal components and trading them as individual zero-coupon securities. This process allows investors to purchase and trade the coupon or principal components separately, providing greater flexibility in managing their investment portfolios.

<div>
    <center>
        <img src="figs/Fig-STRIPS-Schematic.png" width="900"/>
    </center>
</div>

For example, the 5-year Treasury note with annual coupon payments of $C$ USD and a face (par) value of $V_{P}$ (USD) shown above can be stripped into six separate zero-coupon securities, i.e., five zero-coupon bonds, each with face values of $C$ and maturity of $T$= 1,2,3,4 and 5 years, and a six security with face  (par) value of $V_{P}$ USD with a duration of $T$ = 5 years. In the general case, a treasury note or bond with $N=\lambda{T}$ coupon payments, where $T$ denotes the maturity in years, and $\lambda$ represents the number of coupon payments per year, can be stripped into $N+1$ separate zero-coupon securities.

Beyond thier immediate value as investment tools, STRIPS bonds are interesting as they provide another look at the [Term Structure of Interest Rates](https://www.investopedia.com/terms/t/termstructure.asp#:~:text=Essentially%2C%20term%20structure%20of%20interest,current%20state%20of%20an%20economy), i.e., how the change in the [short rates](https://en.wikipedia.org/wiki/Short-rate_model) influences the price and yeild the bond.

## Learning Objectives
In this example, we will create a model for a 30-year treasury bond with semi-annual coupon payments. Then, we will strip the coupon payments and face value to produce 61 zero-coupon STRIPS bonds. We will then show the bootstrapping method, an alternative approach to estimating the short rates from the price of the STRIPS zero-coupon products.

* __Objective 1__: Our first objective is to build a model for a 30-year treasury bond and strip the bond's coupon payments and face value. To make our calculations realistic, we will use coupon and market interest values from recent 30-year bond quotations. We will encounter an interesting challenge when we strip the 30-year bond, which is how to price the zero-coupon STRIP products.
* __Objective 2__: Our second objective is to demonstrate the bootstrapping method to estimate the short rates from the STRIP zero-coupon products.

## Setup
In the following code blocks, we set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this exercise, and custom types to model the components of our problem.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-3`


## Objective 1: STRIP a 30-year treasury bond

In [2]:
# Set parameters for the original 30-year bond
Vₚ,T,r,c,λ = 1000.0,30.0,0.03832,0.03625,2

# Build a discrete compounding model -
discrete_compounding = DiscreteCompoundingModel();

# Build a 3-year model, and compute the price using the short-hand syntax
model = build(MyUSTreasuryCouponSecurityModel, (
            par = Vₚ, T = T, rate = r, coupon = c, λ = λ
        )) |> discrete_compounding;

# show -
println("The 30-year bond with face value Vₚ = $(Vₚ) USD is priced at $(model.price) USD")

The 30-year bond with face value Vₚ = 1000.0 USD is priced at 963.2794794856908 USD


### Build the zero coupon STRIPS from the parent 30-year bond

In [3]:
strips = strip(model);

#### Challenge: How should we price the zero-coupon STRIPS?
[STRIPS trade at a discount to their face value since holders do not receive any additional income through coupon payments](https://www.investopedia.com/terms/s/stripbond.asp#:~:text=How%20Strip%20Bonds%20Are%20Priced,present%20value%2C%20and%20vice%20versa). However, that fraction is set by the person selling the STRIPS onto the secondar treasury market. Let's consider a hypothetical pricing scheme where we charge a changing fraction of the par value:

$$V_{B} = \alpha\cdot{V}_{P}$$

where $\alpha \equiv (0.95)^{T}$ and $T$ denotes the maturity of the zero coupon STRIPS. Now that we have the price, par value and the duration we can compute the yeild to maturity for each zero coupon security 

In [24]:
# initialize -
number_of_zc_bonds = length(strips)

# main loop -
for i ∈ 1:number_of_zc_bonds
    
    # grab the bond model -
    zc_model = strips[i];
    T = zc_model.T
    
    # build -
    α = 0.95^(T)
    
    # set the price -
    Vₚ = zc_model.par;
    zc_model.price = α*Vₚ
    
    # do a YTM calculation -
    yeild = YTM(zc_model, discrete_compounding)
    
    # update the model -
    zc_model.rate = yeild;
end

#### Visualize the first $m$ STRIPS products

In [25]:
number_of_strips_to_view = 10
strips_table_data_array = Array{Any,2}(undef, number_of_strips_to_view, 5);
strips_table_header_array = (["index", "Maturity","Face","Price","Yield"],["","yr","USD","USD","%"]);

for i ∈ 1:number_of_strips_to_view
    
    # get the zc STRIPS model -
    zc_model = strips[i];
    
    # add data to the table data array
    strips_table_data_array[i,1] = i
    strips_table_data_array[i,2] = zc_model.T
    strips_table_data_array[i,3] = zc_model.par
    strips_table_data_array[i,4] = zc_model.price
    strips_table_data_array[i,5] = zc_model.rate    
end

# show the table -
pretty_table(strips_table_data_array, header=strips_table_header_array)

┌───────┬──────────┬────────┬─────────┬───────────┐
│ index │ Maturity │   Face │   Price │     Yield │
│       │       yr │    USD │     USD │         % │
├───────┼──────────┼────────┼─────────┼───────────┤
│     1 │      0.5 │ 18.125 │ 17.6661 │ 0.0526316 │
│     2 │      1.0 │ 18.125 │ 17.2188 │ 0.0526316 │
│     3 │      1.5 │ 18.125 │ 16.7828 │ 0.0526316 │
│     4 │      2.0 │ 18.125 │ 16.3578 │ 0.0526316 │
│     5 │      2.5 │ 18.125 │ 15.9436 │ 0.0526316 │
│     6 │      3.0 │ 18.125 │ 15.5399 │ 0.0526316 │
│     7 │      3.5 │ 18.125 │ 15.1464 │ 0.0526316 │
│     8 │      4.0 │ 18.125 │ 14.7629 │ 0.0526316 │
│     9 │      4.5 │ 18.125 │ 14.3891 │ 0.0526316 │
│    10 │      5.0 │ 18.125 │ 14.0248 │ 0.0526316 │
└───────┴──────────┴────────┴─────────┴───────────┘


## Objective 2: Bootstrapping method to compute the short rates using STRIPS
We can determine the _short rates_, which represent the market rate of interest between periods $j\rightarrow{j+1}$ and are denoted by $r_{j+1,j}$, by analyzing the prices of the various STRIPS zero coupon products based on their maturity. Using a discrete discounting model, the short rates are calculated according.:

$$
\begin{eqnarray}
\frac{V_{P,1}}{V_{B,1}} & = & \left(1+r_{1,0}\right) \\
\frac{V_{P,2}}{V_{B,2}} & = & \left(1+r_{2,1}\right)\cdot\left(1+r_{1,0}\right) \\
\vdots & = & \vdots \\
\frac{V_{P,N}}{V_{B,N}} & = & \prod_{i=1}^{N}\left(1+r_{i,i-1}\right) \\
\end{eqnarray}
$$

where $V_{P,i}$ and $V_{B,i}$ denote the face (par) value and price of the $i^{th}$ zero-coupon bond (both of which are known). Thus, we can solve for $r_{1,0}$, then insert that into the following expression to solve for $r_{2,1}$, and so on. Systematically, we can solve for the log-transformed short rates as a system of linear algebraic equations (LAEs) of the from:

$$
\mathbf{A}\mathbf{x} = \mathbf{b}
$$

where $x_{i} = \log\left(1+r_{i,i-1}\right)$, $b_{i} = \log\left(V_{P,i}/V_{B,i}\right)$ and $\mathbf{A}$ is a lower-triangular matrix of `1`'s.

In [28]:
# initialize -
short_rate_array = Array{Float64,1}(undef, number_of_strips_to_view);
b = Array{Float64,1}(undef, number_of_strips_to_view)
A = Array{Float64,2}(undef, number_of_strips_to_view,number_of_strips_to_view);

# fill the A with zeros -
fill!(A,0.0);

# populate b -
for i ∈ 1:number_of_strips_to_view
    
    # grab the model, and get some data -
    zc_model = strips[i];
    r̄ = zc_model.rate
    T = zc_model.T
    Vₚ = zc_model.par
    Vᵦ = zc_model.price
     
    # compute, add to b -
    b[i] = log10(Vₚ/Vᵦ);
end

# build A -
for i ∈ 1:number_of_strips_to_view
    for j ∈ 1:number_of_strips_to_view
        if (i ≥ j)
            A[i,j] = 1.0
        end
    end
end

# invert A -
x̂ = inv(A)*b;

# compute the short rates -
for i ∈ 1:number_of_strips_to_view
    short_rate_array[i] = 10^(x̂[i]) - 1
end

### Check: Short rates verified using effective market rate of interest $\bar{r}$